In [2]:
import datasets
import torch.nn as nn
import data_process as dp
import torchtext
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

2023-02-14 22:22:27.972104: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 22:22:29.109414: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-14 22:22:29.109562: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-14 22:22:29.109570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
moves_df = datasets.load_sql_to_df("SELECT * FROM english_annotated_moves", "./chess.db")
moves_df.head(5)

,index,position,move,comment,halfmove_number,game_id,sentiment,auto_sentiment
0,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,e2e4,This is my first gameknot game against someone...,1,44,2,0
1,1,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,b1c3,"I've been playing the Vienna Gambit as white, ...",3,44,-1,0
2,2,rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQK...,f8c5,Minor disappointment.,4,44,-1,0
3,3,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2NP4/PPP2PP...,f2f4,"My idea here is to expand on the kingside, dri...",9,44,-1,0
4,4,r1bqk2r/ppp2ppp/3p1n2/n1b1pP2/2B1P3/2NP4/PPP3P...,d1f3,"Maybe this isn't the greatest plan, since with...",13,44,0,0


In [4]:
glove_embbedings = torchtext.vocab.GloVe(name="6B", dim=50)

In [5]:
moves_df = dp.prepare_data_for_sentiment_analysis_training(moves_df, glove_embbedings.stoi)
moves_df

,comment,sentiment
0,"[maybe, this, be, not, the, great, plan, since...",0
1,"[?, too, slow, maybe, .]",0
2,"[?, this, allow, a, combination, by, white, .,...",0
3,"[!, !, brilliant, move, it, give, black, the, ...",1
4,"[!, !, this, move, be, really, fantastic, ., m...",1
...,...,...
19864,"[in, hindsight, be, need, to, get, my, king, t...",0
19865,"[keep, black, alive]",0
19866,"[time, remain, minute, .]",0
19867,"[black, be, win, .]",1


In [6]:
data = list(zip(moves_df.comment, moves_df.sentiment))
datasets.add_padding_vector_to_embeddings(glove_embbedings)
dataset = datasets.PretrainedEmbeddingsIndicesDataset(data, glove_embbedings)

In [7]:
x,y = dataset[5000]

In [8]:
print([glove_embbedings.itos[t.item()] for t in x])
print(x)
print(y)

['this', 'be', 'a', 'excellent', 'move', '!', 'instead', 'of', 'black', 'can', 'go']
tensor([  38,   31,    8, 4346,  484,  806,  774,    4,  522,   87,  243])
tensor(1)


In [9]:
def create_embedding_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer


In [15]:
class SentimentAnalysisLSTM(nn.Module):
    def __init__(self, embeddings: torchtext.vocab.Vectors, hidden_dim, num_layers = 2, dropout = 0.1):
        super().__init__()
        
        self.embedding = create_embedding_layer(embeddings.vectors, non_trainable=True)

        self.lstm = nn.LSTM(embeddings.dim, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True)

        self.fc = nn.Linear(hidden_dim, 1)
        
        
    def forward(self, x):
        print(f"x.shape: {x.shape}")

        embedded = self.embedding(x)

        print(f"embedded.shape: {embedded.shape}")

        lstm_output, _ = self.lstm(embedded)

        print(f"lstm_output.shape: {lstm_output.shape}")
        
        last_lstm_output = lstm_output[:, -1, :]

        print(f"last_lstm_output.shape: {last_lstm_output.shape}")

        output = self.fc(last_lstm_output)

        print(f"output.shape: {output.shape}")

        return output

model = SentimentAnalysisLSTM(glove_embbedings, 128)
model


SentimentAnalysisLSTM(
  (embedding): Embedding(400001, 50)
  (lstm): LSTM(50, 128, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [16]:
batch = torch.tensor([[1,2,3,0,0],[1,2,3,4,5]], dtype=torch.int)
model(batch)

x.shape: torch.Size([2, 5])
embedded.shape: torch.Size([2, 5, 50])
lstm_output.shape: torch.Size([2, 5, 128])
last_lstm_output.shape: torch.Size([2, 128])
output.shape: torch.Size([2, 1])


tensor([[-0.0353],
        [-0.0417]], grad_fn=<AddmmBackward0>)

In [36]:
train_df, val_df = train_test_split(test_df, test_size=0.2, random_state=42)

In [46]:
emb = nn.Embedding(1,2)
emb.requires_grad_()

Embedding(1, 2)